<a href="https://colab.research.google.com/github/Ira-a02/DataAnalysis/blob/main/Hw1_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer


In [ ]:
data=pd.read_csv("salary-train.csv")

In [ ]:
data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [ ]:
import re
def preprocessing_text(text):
    text=re.sub('[^a−zA−Z0−9]', ' ', text.lower())
    return text


In [ ]:
data['FullDescription'] = data['FullDescription'].apply(preprocessing_text)

In [ ]:
vector_features=TfidfVectorizer(min_df=5)
matrix=vector_features.fit_transform(data['FullDscription'] )

In [ ]:
data['LocationNormalized'].fillna('nan', inplace=True)
data['ContractTime'].fillna('nan', inplace=True)

<ipython-input-19-65ef4ded5f13>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['LocationNormalized'].fillna('nan', inplace=True)
<ipython-input-19-65ef4ded5f13>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

In [ ]:
#Transforms lists of feature-value mappings to vectors.
#Learn a list of feature name -> indices mappings and transform X.
#Like fit(X) followed by transform(X), but does not require materializing X in memory.

enc = DictVectorizer()
X_train_categ = enc.fit_transform(data[ [ 'LocationNormalized' , 'ContractTime' ] ].to_dict('records') )
X_test_categ = enc.transform(data[ [ 'LocationNormalized', 'ContractTime'] ].to_dict( 'records') )

In [ ]:
# concatenate sparse matrices with the same number of rows
from scipy.sparse import hstack
X_train=hstack((matrix, X_test_categ))

In [ ]:
print(X_train.shape)

(60000, 1792)


In [ ]:
y_train = data['SalaryNormalized']
ridge_model = Ridge(alpha=1)
ridge_model.fit(X_train, y_train)

Ridge(alpha=1)

In [ ]:
data_mini=pd.read_csv('/content/salary-test-mini.csv')

In [ ]:
data_mini['FullDescription'] = data_mini['FullDescription'].apply(preprocessing_text)

In [ ]:
X_train_mini_matrix=vector_features.transform(data_mini['FullDescription'])
X_train_mini_categ=enc.transform(data_mini[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [ ]:
X_train_mini=hstack([X_train_mini_matrix, X_train_mini_categ])

In [ ]:
y_train_mini=ridge_model.predict(X_train_mini)

In [ ]:
y_train_mini_round=[round (i, 2) for i in y_train_mini]
print(y_train_mini_round)

[33501.43, 29720.68]


In [ ]:
output = " ".join(map(str, y_train_mini_round))
with open('predictions.txt', 'w') as file:
    file.write(output)